---
title: Météo à Montpellier
output:
  quarto::quarto_document:
    css: "style.css"
jupiter: python3
---

Ce site affiche les prévisions météorologiques pour Montpellier, France, en utilisant l'API Open-Meteo. Les données sont mises à jour quotidiennement à 5h00 (UTC) via un workflow GitHub Actions. Le code source est disponible sur [GitHub](https://github.com/jeannemann/Projet-perso.git).

In [ ]:
#| echo: false
# %%
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/meteofrance?latitude=43.6109&longitude=3.8763&daily=weather_code,temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant&timezone=Europe%2FBerlin"
params = {
	"latitude": 43.6109,
	"longitude": 3.8763,
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "precipitation_sum", "wind_speed_10m_max","wind_direction_10m_dominant"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
#print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
#print(f"Elevation {response.Elevation()} m asl")
#print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
#print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(3).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(4).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(5).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant

daily_dataframe = pd.DataFrame(data = daily_data)
#print(daily_dataframe)

def image(daily_weather_code):
  if daily_weather_code == 0:
    return "https://openclipart.org/download/30061/weather-clear.svg"
  if daily_weather_code in [1,2,3]:
    return "https://openclipart.org/download/122887/weather.svg"
  if daily_weather_code in [45,48]:
    return "https://openclipart.org/download/278410/foggy.svg"
  if daily_weather_code in [51,53,55]:
    return "https://upload.wikimedia.org/wikipedia/commons/7/78/Weather-drizzle.svg"
  if daily_weather_code in [56,57]:
    return "bruine glacée"
  if daily_weather_code in [61,63,65,80,81,82]:
    return "https://openclipart.org/download/22004/nicubunu-Weather-Symbols-Rain.svg"
  if daily_weather_code in [66,67]:
    return "pluie glacée"
  if daily_weather_code in [71,73,75,85,86]:
    return "https://openclipart.org/download/189278/snow-flake-5.svg"
  if daily_weather_code == 77:
    return "grains de neige"
  if daily_weather_code in [95,96,99]:
    return "https://openclipart.org/download/217034/thunderstorm.svg" 
  
def dir(daily_wind_direction_10m_dominant):
  if daily_wind_direction_10m_dominant < 45 or daily_wind_direction_10m_dominant >= 315:
    return "https://openclipart.org/download/325034/north-arrow-for-maps.svg"
  if daily_wind_direction_10m_dominant >= 45 and daily_wind_direction_10m_dominant <135 :
    return "https://openclipart.org/download/272502/Arrow4L-250.svg"  
  if daily_wind_direction_10m_dominant >= 135 and daily_wind_direction_10m_dominant < 225 :
    return "https://openclipart.org/download/26525/Soeb-Plain-Arrows-7.svg"
  if daily_wind_direction_10m_dominant >= 225 and daily_wind_direction_10m_dominant < 315 :
    return "https://openclipart.org/download/26523/Soeb-Plain-Arrows-5.svg"


from IPython.display import display,HTML

html_content=f"""
<table>
  <thead>
    <tr>
      <th>Données</th>
      <th>Aujourd'hui</th>
      <th>Demain (J+1)</th>
      <th>J+2</th>
      <th>J+3</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Température max</td>
      <td>{round(daily_temperature_2m_max[0])}°C</td>
      <td>{round(daily_temperature_2m_max[1])}°C</td>
      <td>{round(daily_temperature_2m_max[2])}°C</td>
      <td>{round(daily_temperature_2m_max[3])}°C</td>
    </tr>
    <tr>
      <td>Température min</td>
      <td>{round(daily_temperature_2m_min[0])}°C</td>
      <td>{round(daily_temperature_2m_min[1])}°C</td>
      <td>{round(daily_temperature_2m_min[2])}°C</td>
      <td>{round(daily_temperature_2m_min[3])}°C</td>
    </tr>
    <tr>
      <td>Météo du jour</td>
      <td><img src={image(daily_weather_code[0])} width="50" height="50"></td>
      <td><img src={image(daily_weather_code[1])} width="50" height="50"></td>
      <td><img src={image(daily_weather_code[2])} width="50" height="50"></td>
      <td><img src={image(daily_weather_code[3])} width="50" height="50"></td>
    </tr>
    <tr>
      <td>Somme des précipitations</td>
      <td>{round(daily_precipitation_sum[0])}mm</td>
      <td>{round(daily_precipitation_sum[1])}mm</td>
      <td>{round(daily_precipitation_sum[2])}mm</td>
      <td>{round(daily_precipitation_sum[3])}mm</td>
    </tr>
    <tr>
      <td>Vitesse du vent</td>
      <td><img src={dir(daily_wind_direction_10m_dominant[0])} width="10" height="10"> {round(daily_wind_speed_10m_max[0])}km/h</td>
      <td><img src={dir(daily_wind_direction_10m_dominant[0])} width="10" height="10"> {round(daily_wind_speed_10m_max[1])}km/h</td>
      <td><img src={dir(daily_wind_direction_10m_dominant[0])} width="10" height="10"> {round(daily_wind_speed_10m_max[2])}km/h</td>
      <td><img src={dir(daily_wind_direction_10m_dominant[0])} width="10" height="10"> {round(daily_wind_speed_10m_max[3])}km/h</td>
    </tr>
  </tbody>
</table>
"""
display(HTML(html_content))

Site créé